# Convolutional Neural Network Example - tf.layers API

In [1]:
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Training Parameters
learning_rate = 0.001 #learning rate
num_steps = 2000 # train steps
batch_size = 128 # size

# Network Parameters
num_input = 784 # MNIST img size (img shape: 28*28)
num_classes = 10 # MNIST classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units = (1-p)

In [3]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator
        x = x_dict['images']

        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # conv layer, 32 kernels, size 5x5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # max pooling layer, stripe 2 
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # conv layer, 64 kernels, size 3x3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # max pooling layer, stripe 2 
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # fully connected layer, convert to one layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # fully connected layer
        fc1 = tf.layers.dense(fc1, 1024)
        # apply Dropout (when training is ON, when testing is OFF)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # output layer
        out = tf.layers.dense(fc1, n_classes)

    return out

In [4]:
# TF Estimator
def model_fn(features, labels, mode):
    
    # construct NN
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # predict
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # loss function and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # model accuracy
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators return EstimatorSpec
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [5]:
# construct Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Peter\\AppData\\Local\\Temp\\tmpdy_fuj8g', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [6]:
# training input function
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

# start training
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Peter\AppData\Local\Temp\tmpdy_fuj8g\model.ckpt.
INFO:tensorflow:loss = 2.32756, step = 1
INFO:tensorflow:global_step/sec: 10.8217
INFO:tensorflow:loss = 0.271779, step = 101 (9.242 sec)
INFO:tensorflow:global_step/sec: 11.2508
INFO:tensorflow:loss = 0.149701, step = 201 (8.888 sec)
INFO:tensorflow:global_step/sec: 11.2615
INFO:tensorflow:loss = 0.159865, step = 301 (8.880 sec)
INFO:tensorflow:global_step/sec: 11.5177
INFO:tensorflow:loss = 0.0478833, step = 401 (8.682 sec)
INFO:tensorflow:global_step/sec: 11.1845
INFO:tensorflow:loss = 0.0932522, step = 501 (8.941 sec)
INFO:tensorflow:global_step/sec: 11.1908
INFO:tensorflow:loss = 0.152593, step = 601 (8.936 sec)
INFO:tensorflow:global_step/sec: 11.3429
INFO:tensorflow:loss = 0.0341131, step = 701 (8.816 sec)
INFO:tensorflow:global_step/sec: 11.5411
INFO:tensorflow:loss = 0.0116405, step = 801 (8.665 sec)
INFO:tensorflow:global_step/sec

In [ ]:
# eval model
# the inputs
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
model.evaluate(input_fn)

In [ ]:
# predict single image
n_images = 4

# test images
test_images = mnist.test.images[:n_images]

# prepare input data
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_images}, shuffle=False)

# predict
preds = list(model.predict(input_fn))

# plotting
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction:", preds[i])